In [ ]:
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
os.chdir('/local/home/dhaziza/entrack')
sys.path.append('/local/home/dhaziza/entrack/')
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="4"


import glob
import random
import nibabel as nib
from src.data.plots import display_image_path

In [ ]:
def display_all(gl, filter_shape=None):
    print(gl)
    files = glob.glob(gl)
    all_data = []
    for f in files:
        #f = random.choice(files)
        dat = display_image_path(f)
        if filter_shape is not None and list(filter_shape) != list(dat.shape):
            continue
        all_data.append(dat)
    all_data = np.array(all_data)
    # Percentile
    ptiles_x = np.linspace(0, 100, num=100)
    ptiles = np.percentile(all_data, ptiles_x)
    plt.plot(ptiles_x, ptiles)
    plt.title("Intensity percentiles")
    plt.xlabel("%")
    plt.ylabel("Percentile value")
    plt.show()
    for p in range(0, 100, 10):
        print('Percentile %f%%: %f' % (p, ptiles[p]))
    plt.plot(ptiles_x, np.log(ptiles+10))
    plt.title("Intensity percentiles (log scale)")
    plt.xlabel("%")
    plt.ylabel("log(10 + percentile value)")
    plt.show()
    # Histogram
    hist_bins = 100
    hist_range = (0, 1000)
    hgram, _ = np.histogram(all_data, bins=hist_bins, range=hist_range)
    plt.plot(
        np.linspace(hist_range[0], hist_range[1], num=hist_bins),
        [[v, 0] for v in hgram]
    )
    plt.title("Intensity Histogram")
    plt.xlabel("Voxel intensity")
    plt.ylabel("Count")
    plt.show()

#_ = display_all('data/raw/KOLN_T1/*/*/*.nii.gz', [256, 256, 128])
#display_all('/local/ADNI_AIBL/ADNI_AIBL_T1_smoothed/ADNI/*.nii.gz')
#display_all('/local/ADNI_AIBL/ADNI_AIBL_T1_normalized/train/*.nii.gz')
#display_all('/local/OASIS/OASIS_normalized/CV1/*.nii.gz')
_ = display_image_path('data/raw/templates/MNI152_T1_1mm_brain.nii.gz')
_ = display_image_path('data/raw/templates/MNI152_T2_1mm_brain.nii.gz')
#display_all('/local/KOLN/T2/02_registered/*.nii.gz')


In [ ]:
def debug_images_ppmi(ids=['*']):
    for id in ids:
        if id != '*':
            for f in glob.glob('/local/PPMI/raw/*/*/*/*/*I%s.nii' % id):
                print(f)
                display_image_path(f)
        for f in glob.glob('/local/PPMI/01*/I%s.nii.gz' % id):
            if len(f.split('/')[-1].split('_')) > 1:
                continue
            print(f)
            display_image_path(f)
        for f in glob.glob('/local/PPMI/12*/I%s.nii.gz' % id):
            if len(f.split('/')[-1].split('_')) > 1:
                continue
            print(f)
            display_image_path(f)

def debug_display_ppmi():
    print('PPMI')
    for f in glob.glob('/local/PPMI/02*/*.nii.gz'):
        if len(f.split('/')[-1].split('_')) > 1:
            continue
        print(f)
        display_image_path(f)


def debug_display_koln_t2(ids=['*']):
    print('KOLN/T2')
    for img in ids:
        if img != '*':
            for f in glob.glob('/local/KOLN/T2/raw/*/%s/*.nii.gz' % img):
                print(f)
                display_image_path(f)
        for f in glob.glob('/local/KOLN/T2/01*/I%s.nii.gz' % img):
            if len(f.split('/')[-1].split('_')) > 1:
                continue
            print(f)
            display_image_path(f)

def debug_display_adni_t2():
    for f in glob.glob('/local/ADNI_AIBL/ADNI_AIBL_T2_smoothed/*.nii.gz'):
        print(f)
        display_image_path(f)
        
PPMI_WRONG = [498879, 369009, 769275, 430154, 436355, 901139, 491582, 368565]
PPMI_WRONG = [366339, 373999, 340757, 249413, 364028, 366334, 223787, 366361, 382297, 378210]
KOLN_BUGGED = [150622, 111104, 150706, 120210]
#print('rm ' + ' '.join(['/local/PPMI/01_brain_extracted/I%d.nii.gz' % i for i in PPMI_WRONG]))
debug_images_ppmi([713705])
#debug_display_koln_t2()
#debug_display_adni_t2()

In [ ]:
import subprocess
import os


def koln_data(img):
    return '/local/KOLN/T2/raw/*/%s/*.nii.gz' % img
def ppmi_data(img):
    return '/local/PPMI/raw/*/*/*/*/*I%s.nii' % img

def extract_and_display(id_list, id_to_file, options_list):
    print('------------- extract_and_display -------------')
    for idx, img in enumerate(id_list):
        print('------------- image %s' % (img))
        raw_file = glob.glob(id_to_file(img))
        assert(len(raw_file) > 0)
        if len(raw_file) > 1:
            print('[WARNING] Several files with this id. Taking the first one (%s)' % raw_file[0])
        raw_file = raw_file[0]
        #orig_data = display_image_path(raw_file)
        #print(orig_data.shape)
        for options in options_list:
            mri_output = '/tmp/bet_test/%s.nii.gz' % img
            cmd = 'bet {mri_image} {mri_output} {options}'
            cmd = cmd.format(
                mri_image=raw_file,
                mri_output=mri_output,
                options=options,
            )
            os.environ['FSLOUTPUTTYPE'] = 'NIFTI_GZ'
            os.environ['FSLDIR'] = '/local/fsl'
            print(cmd)
            subprocess.call(cmd, shell=True)
            display_image_path(mri_output)

#extract_and_display(KOLN_BUGGED, koln_data, ['-f 0.5'])
extract_and_display(PPMI_WRONG, ppmi_data, ['-f 0.1', '-S -f 0.8', '-B -f 0.8'])

In [ ]:
extract_and_display(PPMI_WRONG, ppmi_data, ['-f 0.1', '-f 0.15', '-f 0.2'])

In [ ]:
extract_and_display(PPMI_WRONG, ppmi_data, ['-f 0.1', '-R -f 0.1', '-S -f 0.1', '-B -f 0.1', '-A -f 0.1'])